In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
from sklearn.linear_model import LinearRegression

## Numerical parameters

In [2]:
density = 1026 #Water density - kg/m3
c_p = 3850 #Specific heat of salt water - J/kgC
seconds = 30*24*60*60 #Time in a month - seconds

# Auto Regression Model

In [3]:
class AutoRegression:
    """
    Class for auto-regression models!
     
    Built with help from https://medium.com/@karen.mossoyan/building-an-autoregression-model-for-time-series-analysis-in-python-49402bdd6d08
    """
    def __init__(self):
        self.order = 1 # Order of model
        self.model = LinearRegression() # Type of model
        self.sigma = None # Generate noise
        
    def generate_train_x(self, X):
        """
        Data to be trained on
        
        """
        n = len(X) 
        
        # Storing values X_1 through X_(n-p) in a column
        ans = X[:n-self.order]
        ans = np.reshape(ans, (-1,1))
        
        return ans
    
    def generate_train_y(self, X):
        """
        The target variable
        
        """
        return X[self.order:]
    
    def fit(self, X):
        """
        Assembly
        
        """
        self.sigma = np.std(X)
        train_x = self.generate_train_x(X)
        train_y = self.generate_train_y(X)
        self.model.fit(train_x, train_y)
        
    def predict(self, X, num_predictions, mc_depth):
        """
        Predicts an arbitrary amount of steps into the future and performs a Monte Carlo Simulation.
        
        Variables:
            num_predictions - amount of steps we'll be predicting
            mc_depth - amount of MC simulations of the model we'll be averaging
        """
        X = np.array(X)
        ans = np.array([])
        
        for j in range(mc_depth): 
            ans_temp = []
            a = X[-self.order:] # The last value of the given signal
            
        # Predicting using a, the trained model, and noise
        for i in range(num_predictions):
            new = self.model.predict(np.reshape(a, (1,-1)) + np.random.normal(loc=0, scale=self.sigma)) 
            ans_temp.append(new)
            # Dropping the oldest value...
            a = np.roll(a, -1)
            # ...and putting the newest one in the front
            a[-1] = new
            
        if j==0:
            ans = np.array(ans_temp)
        else:
            ans += np.array(ans_temp)
            
        # Divide by the amount of MC Simulations we're running!    
        ans /= mc_depth
        
        return ans

# Integration